<center>

**CURVAS ELIPTICAS**

</center>

<p align="center">
    <img src="https://logowik.com/content/uploads/images/escudo-de-la-universidad-nacional-de-colombia-20163327.logowik.com.webp" width="400">
</p>

<center>

# **⛎Curvas Elípticas♉**

<p align="center">
    <img src="https://sslinsights.com/wp-content/uploads/2024/06/ecc-vs-rsa-comparison.png" width="600">
</p>

<center>

<div align="justify">



**Intercambio de clave con ECC (ECDH)**

1. **Parámetros de la curva**  
   - Se trabaja sobre la curva definida por  
     $$y^2 = x^3 + a\,x + b \pmod{p}$$  
     con $p$ primo y parámetros $(a,b)$ elegidos de antemano.  
   - Se dispone de un punto generador $G$ de orden $n$ en la curva.

2. **Claves privadas**  
   - Alice elige un entero aleatorio $d_A\in[1,n-1]$.  
   - Bob elige un entero aleatorio $d_B\in[1,n-1]$.

3. **Claves públicas**  
   - Alice calcula  
     $$Q_A = d_A\,G.$$  
   - Bob calcula  
     $$Q_B = d_B\,G.$$

4. **Cálculo del secreto compartido**  
   - Alice recibe $Q_B$ y calcula  
     $$S = d_A\,Q_B = d_A\,d_B\,G.$$  
   - Bob recibe $Q_A$ y calcula  
     $$S = d_B\,Q_A = d_B\,d_A\,G.$$  
   Ambos obtienen el mismo punto $S=(x_S,y_S)$ sin exponer sus claves privadas.

---

**Derivación de la clave AES**

- Se extrae la coordenada $x_S$ del punto compartido $S$ y se aplica SHA‑256:  
  $$K_{\text{AES}} = \mathrm{SHA256}\bigl(\textrm{str}(x_S)\bigr)_{[0\ldots15]}$$  
  de modo que obtenemos una clave de 128 bits.

---

**Cifrado y descifrado con AES‑CBC**


1. **Cifrado**  
   - Se genera un vector de inicialización (IV) aleatorio de 16 bytes.  
   - Se aplica AES‑CBC con relleno PKCS#7 al mensaje \(M\).  
   - El IV y el texto cifrado \(C\) se codifican en hexadecimal y en Base64 para su transporte.

 $$
 C = \mathsf{AES\text{-}CBC}_{K_{\mathrm{AES}},\,IV}\bigl(\mathrm{pad}(M)\bigr)
 $$

2. **Descifrado**  
   - Usando la misma clave \(K_{\mathrm{AES}}\) y el IV recibido, se recupera el mensaje.  
   - Si el relleno no es válido o la clave es incorrecta, la operación falla.

 $$
 M = \mathrm{unpad}\!\Bigl(\mathsf{AES\text{-}CBC}^{-1}_{\,K_{\mathrm{AES}},\,IV}(C)\Bigr)
 $$
---

**Interfaz interactiva**

- Se presenta un panel de chat donde:
  1. **Alice** y **Bob** comparten en pantalla su clave pública y el secreto derivado.  
  2. Cada mensaje se cifra con AES‑CBC antes de “enviarse” y se muestra:
     - Mensaje original
     - IV en **hex**
     - Texto cifrado en **hex** y en **Base64**
     - Texto resultante tras el descifrado

- De este modo, se ilustra en tiempo real cómo:
  - **ECDH** garantiza que solo Alice y Bob conocen $K_{\text{AES}}$.  
  - **AES‑CBC** ofrece confidencialidad de los mensajes intercambiados.

---

**Seguridad y recomendaciones**

- En un entorno real, usar curvas estándar de al menos 256 bits (p. ej. **prime256v1**, **secp256k1**) y generar claves con un CSPRNG fiable.  
- Nunca reutilizar IVs con la misma clave.  
- Proteger las implementaciones de ataques de canal lateral y validar siempre que las claves públicas recibidas pertenezcan a la curva.  

</div>

<center>

**📥Instalación de paquetes📦**

In [ ]:
!pip install ipywidgets pycryptodome


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 75.3 MB/s eta 0:00:00


**📥Importaciones📦**

In [ ]:

from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from hashlib import sha256
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
from sympy import mod_inverse
import base64

**👨‍💻Implementación👩‍💻**

In [ ]:

p = 17
a = 2
b = 2
G = (5, 1)
n = 19

def ecc_add(P, Q):
    if P is None: return Q
    if Q is None: return P
    if P == Q:
        if P[1] == 0: return None
        m = ((3 * P[0]**2 + a) * mod_inverse(2 * P[1], p)) % p
    else:
        if P[0] == Q[0]: return None
        m = ((Q[1] - P[1]) * mod_inverse(Q[0] - P[0], p)) % p
    x = (m**2 - P[0] - Q[0]) % p
    y = (m * (P[0] - x) - P[1]) % p
    return (x, y)

def ecc_mul(k, P):
    R = None
    for _ in range(k):
        R = ecc_add(R, P)
    return R

def derive_key(point):
    return sha256(str(point[0]).encode()).digest()[:16]

def aes_encrypt(msg, key):
    cipher = AES.new(key, AES.MODE_CBC)
    ct = cipher.encrypt(pad(msg.encode(), AES.block_size))
    return (
        base64.b64encode(cipher.iv).decode(),
        base64.b64encode(ct).decode(),
        cipher.iv.hex(),
        ct.hex()
    )

def aes_decrypt(iv_b64, ct_b64, key):
    try:
        iv = base64.b64decode(iv_b64)
        ct = base64.b64decode(ct_b64)
        cipher = AES.new(key, AES.MODE_CBC, iv)
        return unpad(cipher.decrypt(ct), AES.block_size).decode()
    except:
        return None

# === Claves ECC ===
alice_priv = 5
bob_priv = 7
alice_pub = ecc_mul(alice_priv, G)
bob_pub = ecc_mul(bob_priv, G)
shared_point = ecc_mul(alice_priv, bob_pub)
aes_key = derive_key(shared_point)


chat_output = widgets.Output(layout={
    'border': '1px solid gray',
    'padding': '10px',
    'height': '400px',
    'overflow': 'auto'
})
msg_input = widgets.Text(placeholder="Escribe un mensaje...", layout=widgets.Layout(width='400px'))
send_as_alice = widgets.Button(description="Enviar como Alice", button_style="success", layout=widgets.Layout(width='200px'))
send_as_bob = widgets.Button(description="Enviar como Bob", button_style="info", layout=widgets.Layout(width='200px'))

# === Enviar mensajes ===
def send_message(sender):
    msg = msg_input.value.strip()
    if not msg:
        return
    iv_b64, ct_b64, iv_hex, ct_hex = aes_encrypt(msg, aes_key)
    decrypted = aes_decrypt(iv_b64, ct_b64, aes_key)

    with chat_output:
        sender_label = "🟢 Alice" if sender == "Alice" else "🔵 Bob"
        display(HTML(f"""
        <div style="margin-bottom:15px;">
            <b>{sender_label}</b><br>
            <b>Mensaje original:</b> {msg}<br>
            <b>IV (hex):</b> <code>{iv_hex}</code><br>
            <b>Texto cifrado (hex):</b> <code>{ct_hex}</code><br>
            <b>Texto cifrado (base64):</b> <code>{ct_b64}</code><br>
            <b>Texto descifrado:</b> <span style="color:green;">{decrypted}</span>
        </div>
        """))
    msg_input.value = ""

send_as_alice.on_click(lambda b: send_message("Alice"))
send_as_bob.on_click(lambda b: send_message("Bob"))

# === Mostrar panel ===
display(HTML("<h2 style='color:#00cc99;'>💬 Chat Seguro con ECC + AES</h2>"))
display(HTML(f"""
<p><b>🔐 Clave AES derivada:</b> <code>{aes_key.hex()}</code></p>
<p><b>📍 Punto ECC compartido:</b> {shared_point}</p>
<p><b>🔑 Clave privada/pública Alice:</b> {alice_priv} / {alice_pub}</p>
<p><b>🔑 Clave privada/pública Bob:</b> {bob_priv} / {bob_pub}</p>
<hr>
"""))
display(chat_output)
display(widgets.HBox([msg_input, send_as_alice, send_as_bob]))


Output(layout=Layout(border='1px solid gray', height='400px', overflow='auto', padding='10px'))